# Flow
1. 추론 대상 이미지를 받아온다.
2. 추론한다.
3. detected 이미지와 bbox 정보를 얻는다.
4. 이미지와 bbox를 저장한다.

In [1]:
import os
import re
import torch
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

In [2]:
## CONSTANTS ##
IMAGE_SIZE = 640
IMAGE_PATH = 'wheel_origin/'  # images from
BOLT_PATH = 'bolt_images/'   # images to
MODEL_PATH = 'weights/'
NORMAL_BBOXES_PATH = 'detection_normal/'
PROBLEM_BBOXES_PATH = 'detection_problem/'
REGEX = re.compile('.jpg|.png|.jpeg|.gif')

In [3]:
def load_detection_model(model_name, model_path=MODEL_PATH):
    path = model_path + model_name
    return torch.hub.load('.', 'custom', path=path, source='local')

model = load_detection_model('best.pt')

YOLOv5  2023-4-18 Python-3.9.13 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1060, 6144MiB)



requirements: C:\Users\SSAFY\dev\pjt3\detection\requirements.txt not found, check failed.


In [4]:
## define preprocessing function ##
def preprocess_image(image, image_size=IMAGE_SIZE, interpolation=Image.LANCZOS):
    resized = image.resize((image_size, image_size), interpolation)
    b = BytesIO()
    resized.save(b, format='jpeg')
    return Image.open(b)

In [5]:
## define bolt detection model ##
def detect_bolt(model=model, target_image=None, image_path=IMAGE_PATH, bolt_path=BOLT_PATH):
    
    image_list = []

    # 특정 이미지만 추론할 경우
    if target_image:
        image_list.append(target_image)
    # image_path 내 모든 이미지를 추론할 경우
    else:
        image_list = os.listdir(image_path)

    for image in image_list:
        now_image = Image.open(image_path + image)
        now_image = preprocess_image(now_image)

        result = model(now_image)

        # left, upper, right, lower
        bboxes = result.xyxy[0].tolist()
        n_bboxes = len(bboxes)
        
        # left, upper, width, height (normalized)
        bboxes_norm = result.xywhn[0].tolist()
        image_name = re.sub(REGEX, '', image)

        # crop 이미지 저장
        for i in range(n_bboxes):
            now_bbox = bboxes[i]

            x_min = now_bbox[0]
            y_min = now_bbox[1]
            x_max = now_bbox[2]
            y_max = now_bbox[3]

            cropped = now_image.crop((x_min, y_min, x_max, y_max))
            cropped.save('{}{}_bolt_{}.jpg'.format(BOLT_PATH, image_name, i))
        
        # bboxes 저장
        if n_bboxes < 11:
            label_path = PROBLEM_BBOXES_PATH
        else:
            label_path = NORMAL_BBOXES_PATH

        with open(label_path + image_name + '.txt', 'w') as f:
            for bbox in bboxes_norm:
                bbox_str = '{} {} {} {} {}'.format(int(bbox[-1]), bbox[0], bbox[1], bbox[2], bbox[3])
                f.write(bbox_str + '\n')
            f.close()

In [6]:
detect_bolt(model)

TypeError: conv2d() received an invalid combination of arguments - got (JpegImageFile, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!JpegImageFile!, !Parameter!, !NoneType!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!JpegImageFile!, !Parameter!, !NoneType!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
